In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("ml_ready_real_estate_data.csv")
print(df.head())

In [ ]:
#price per postcode / habitable surface
import pandas as pd
import scipy.stats as stats
from sklearn import preprocessing
df = pd.read_csv("ml_ready_real_estate_data.csv")

# Create postcode to location mapping
postcode_to_location = {
    5362: 'Hamois',
    8300: 'Knokke-Heist',
    4983: 'Basse-Bodeux',
    3000: 'Leuven',
    8301: 'Heist-aan-zee',
    3001:'LeuvenBis',
    9830:'Sint-martens-latem',
    3631:'Boorsem',
    2000:'Antwerpen',
    6666:'Wibrin'


}

df['price_per_sqm'] = df['price'] / df['habitableSurface']
#df.groupby('postCode')['price_per_sqm'].mean().sort_values(ascending=False).head(10).plot(kind='bar')
group_data = df.groupby('postCode')['price_per_sqm'].agg(['mean', 'std'])
first_ten = group_data.sort_values('mean', ascending=False).head(10)
labels = [postcode_to_location.get(postcode, str(postcode)) for postcode in first_ten.index]
# Plot with custom labels
ax = first_ten['mean'].plot(kind='bar', yerr=first_ten['std'], capsize=4, 
                           title='Top 10 postcodes by mean price per sqm')
ax.set_xticklabels(labels, rotation=45, ha='right')

plt.ylabel('Price per sqm')
plt.xlabel('Location')
plt.tight_layout()
plt.show()

In [ ]:
#how much add each "has" to price
feature_cols = ['hasSwimmingPool_encoded', 'hasFireplace_encoded', 'hasBasement_encoded', 'hasDressingRoom_encoded']
df[feature_cols + ['price']].corr()['price'].sort_values(ascending=False).plot(kind='bar')
plt.rcParams.update({'font.size': 10})

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

type_mapping = {
    "APARTMENT": 1,
    "HOUSE": 2,
}

# Better reverse mapping - prioritize the underscore versions
reverse_encoding = {}
for name, number in type_mapping.items():
    if number not in reverse_encoding or '_' in name:
        reverse_encoding[number] = name

# Price per sqm
filtered_df = df[(df['habitableSurface'] > 0) & (df['price'] > 0)]
filtered_df['price_per_sqm'] = filtered_df['price'] / filtered_df['habitableSurface']

# Remove outliers
filtered_df = filtered_df[filtered_df['price_per_sqm'] < 10000]
types_house = [1, 2]
filtered_df = filtered_df[filtered_df['type_encoded'].isin(types_house)]

plt.figure(figsize=(16, 8))

# Option 1: Use distinct manual colors
colors = ['#0000FF', '#009900']  # Red and Teal
color_map = {1: colors[0], 2: colors[1]}
point_colors = [color_map[type_code] for type_code in filtered_df['type_encoded']]

# Main scatter plot with manual colors
scatter = plt.scatter(filtered_df['habitableSurface'], filtered_df['price_per_sqm'], 
                     c=point_colors, alpha=0.6)

plt.xscale('log')
plt.grid(True)
plt.xlabel('Habitable Surface (sqm)')
plt.ylabel('Price per sqm (€)')
plt.title('Price per Square Meter vs Surface Area by Property type')

# Create legend with the same colors
unique_types = sorted(filtered_df['type_encoded'].unique())
patches = []
for type_code in unique_types:
    type_color = color_map[type_code]
    name = reverse_encoding.get(type_code, f"Unknown ({type_code})")
    how_to_name = name.replace('_', ' ').title()
    label = f"{type_code}: {how_to_name}"
    patches.append(mpatches.Patch(color=type_color, label=label))
plt.legend(handles=patches)
plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

subtype_mapping = {
    "APARTMENT": 1,
    "HOUSE": 2,
    "FLAT_STUDIO": 3,
    "FLATSTUDIO": 3,
    "DUPLEX": 4,
    "PENTHOUSE": 5,
    "GROUND_FLOOR": 6,
    "GROUNDFLOOR": 6,
    "APARTMENT_BLOCK": 7,
    "APARTMENTBLOCK": 7,
    "MANSION": 8,
    "EXCEPTIONAL_PROPERTY": 9,
    "EXCEPTIONALPROPERTY": 9,
    "MIXED_USE_BUILDING": 10,
    "MIXEDUSEBUILDING": 10,
    "TRIPLEX": 11,
    "LOFT": 12,
    "VILLA": 13,
    "TOWN_HOUSE": 14,
    "TOWNHOUSE": 14,
    "CHALET": 15,
    "MANOR_HOUSE": 16,
    "MANORHOUSE": 16,
    "SERVICE_FLAT": 17,
    "SERVICEFLAT": 17,
    "KOT": 18,
    "FARMHOUSE": 19,
    "BUNGALOW": 20,
    "COUNTRY_COTTAGE": 21,
    "COUNTRYCOTTAGE": 21,
    "OTHER_PROPERTY": 22,
    "OTHERPROPERTY": 22,
    "CASTLE": 23,
    "PAVILION": 24,
}

# Better reverse mapping - prioritize the underscore versions
reverse_encoding = {}
for name, number in subtype_mapping.items():
    if number not in reverse_encoding or '_' in name:
        reverse_encoding[number] = name

# Price per sqm
filtered_df = df[(df['habitableSurface'] > 0) & (df['price'] > 0)]
filtered_df['price_per_sqm'] = filtered_df['price'] / filtered_df['habitableSurface']

# Remove outliers
filtered_df = filtered_df[filtered_df['price_per_sqm'] < 10000]
subtypes_house = [2, 20, 23, 21, 13, 16, 19, 9, 10, 8, 22, 24]
filtered_df = filtered_df[~filtered_df['subtype_encoded'].isin(subtypes_house)]

plt.figure(figsize=(16, 8))

# Main scatter plot
scatter = plt.scatter(filtered_df['habitableSurface'], filtered_df['price_per_sqm'], 
                     c=filtered_df['subtype_encoded'], alpha=0.6, cmap='tab10')

plt.xscale('log')
plt.grid(True)
plt.xlabel('Habitable Surface (sqm)')
plt.ylabel('Price per sqm (€)')
plt.title('Price per Square Meter vs Surface Area by Appartment subtype')

# Create legend
unique_subtypes = sorted(filtered_df['subtype_encoded'].unique())
cmap = plt.cm.tab10
norm = plt.Normalize(vmin=min(unique_subtypes), vmax=max(unique_subtypes))

# Create patches for legend
patches = []
for subtype in unique_subtypes:
    subtype_color = cmap(norm(subtype))
    name = reverse_encoding.get(subtype, f"Unknown ({subtype})")
    how_to_name = name.replace('_', ' ').title()
    label = f"{subtype}: {how_to_name}"
    patches.append(mpatches.Patch(color=subtype_color, label=label))
#plt.rcParams.update({'font.size': 20})
plt.legend(handles=patches)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

subtype_mapping = {
    "APARTMENT": 1,
    "HOUSE": 2,
    "FLAT_STUDIO": 3,
    "FLATSTUDIO": 3,
    "DUPLEX": 4,
    "PENTHOUSE": 5,
    "GROUND_FLOOR": 6,
    "GROUNDFLOOR": 6,
    "APARTMENT_BLOCK": 7,
    "APARTMENTBLOCK": 7,
    "MANSION": 8,
    "EXCEPTIONAL_PROPERTY": 9,
    "EXCEPTIONALPROPERTY": 9,
    "MIXED_USE_BUILDING": 10,
    "MIXEDUSEBUILDING": 10,
    "TRIPLEX": 11,
    "LOFT": 12,
    "VILLA": 13,
    "TOWN_HOUSE": 14,
    "TOWNHOUSE": 14,
    "CHALET": 15,
    "MANOR_HOUSE": 16,
    "MANORHOUSE": 16,
    "SERVICE_FLAT": 17,
    "SERVICEFLAT": 17,
    "KOT": 18,
    "FARMHOUSE": 19,
    "BUNGALOW": 20,
    "COUNTRY_COTTAGE": 21,
    "COUNTRYCOTTAGE": 21,
    "OTHER_PROPERTY": 22,
    "OTHERPROPERTY": 22,
    "CASTLE": 23,
    "PAVILION": 24,
}

# Better reverse mapping - prioritize the underscore versions
reverse_encoding = {}
for name, number in subtype_mapping.items():
    if number not in reverse_encoding or '_' in name:
        reverse_encoding[number] = name

# Price per sqm
filtered_df = df[(df['habitableSurface'] > 0) & (df['price'] > 0)]
filtered_df['price_per_sqm'] = filtered_df['price'] / filtered_df['habitableSurface']

# Remove outliers
filtered_df = filtered_df[filtered_df['price_per_sqm'] < 10000]
subtypes_house = [20, 23, 21, 13, 16, 19, 9, 10, 8, 22, 24]
filtered_df = filtered_df[filtered_df['subtype_encoded'].isin(subtypes_house)]

plt.figure(figsize=(16, 8))

# Main scatter plot
scatter = plt.scatter(filtered_df['habitableSurface'], filtered_df['price_per_sqm'], 
                     c=filtered_df['subtype_encoded'], alpha=0.6, cmap='tab10')

plt.xscale('log')
plt.grid(True)
plt.xlabel('Habitable Surface (sqm)')
plt.ylabel('Price per sqm (€)')
plt.title('Price per Square Meter vs Surface Area by House subtype')

# Create legend
unique_subtypes = sorted(filtered_df['subtype_encoded'].unique())
cmap = plt.cm.tab10
norm = plt.Normalize(vmin=min(unique_subtypes), vmax=max(unique_subtypes))

# Create patches for legend
patches = []
for subtype in unique_subtypes:
    subtype_color = cmap(norm(subtype))
    name = reverse_encoding.get(subtype, f"Unknown ({subtype})")
    how_to_name = name.replace('_', ' ').title()
    label = f"{subtype}: {how_to_name}"
    patches.append(mpatches.Patch(color=subtype_color, label=label))
#plt.rcParams.update({'font.size': 20})
plt.legend(handles=patches)
plt.tight_layout()
plt.show()

In [ ]:
#price by number of bedroom
df['price_range'] = pd.cut(df['price'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
pd.crosstab(df['bedroomCount'], df['price_range']).plot(kind='bar', stacked=True)

In [ ]:
df['bath_bed_ratio'] = df['bathroomCount'] / df['bedroomCount']
plt.figure(figsize=(15, 6))
df.groupby('bath_bed_ratio')['price'].mean().plot(kind='bar')
plt.title('Average Price by Bathroom/Bedroom Ratio')
plt.xlabel('Bathroom/Bedroom Ratio')
plt.ylabel('Price (€)')
plt.xticks(rotation=90)  # Vertical
plt.tight_layout()

In [ ]:
#Price distribution by number of parking space
df.boxplot(column='price', by='totalParkingCount', figsize=(12, 6))
plt.title('Price Distribution by Number of Parking Spaces')
plt.ylabel('Price (€)')

In [ ]:
#How much adding all "luxury features" add to the price
df['luxury_score'] = df['hasSwimmingPool_encoded'] + df['hasFireplace_encoded'] + df['hasBasement_encoded'] + df['hasDressingRoom_encoded']
df.groupby('luxury_score')['price'].mean().plot(kind='bar')
plt.title('Average Price by Luxury Features Count')

In [ ]:
#price per postcode / habitable surface
import pandas as pd
import scipy.stats as stats
from sklearn import preprocessing
df = pd.read_csv("ml_ready_real_estate_data.csv")

# Create province to location mapping
province_to_location = {
    "Brussels": 1,
    "Luxembourg": 2,
    "Antwerp": 3,
    "FlemishBrabant": 4,
    "EastFlanders": 5,
    "WestFlanders": 6,
    "Liège": 7,
    "WalloonBrabant": 8,
    "Limburg": 9,
    "Namur": 10,
    "Hainaut": 11,
    
}
location_to_province = {v: k for k, v in province_to_location.items()}
df['price_per_sqm'] = df['price'] / df['habitableSurface']
#df.groupby('postCode')['price_per_sqm'].mean().sort_values(ascending=False).head(10).plot(kind='bar')
group_data = df.groupby('province_encoded')['price_per_sqm'].agg(['mean', 'std'])
first_ten = group_data.sort_values('mean', ascending=False)
labels = [location_to_province.get(province_encoded, str(province_encoded)) for province_encoded in first_ten.index]
# Plot with custom labels
ax = first_ten['mean'].plot(kind='bar', yerr=first_ten['std'], capsize=4, 
                           title='Province ranked by mean price per sqm')
ax.set_xticklabels(labels, rotation=45, ha='right')

plt.ylabel('Price per sqm')
plt.xlabel('Province')
plt.tight_layout()


#Number of house by province. ( find the Number of each province in our code)
#df.groupby('province_encoded')['price'].median().sort_values(ascending=False).plot(kind='bar')

plt.show()


In [ ]:
# price vs room efficiency
df['room_efficiency'] = (df['bedroomCount'] + df['bathroomCount']) / df['habitableSurface'] * 100
plt.scatter(df['room_efficiency'], df['price'], alpha=0.5)
plt.title('Price vs Room Efficiency (rooms per 100 sqm)')

In [ ]:
df['toilet_efficiency'] = df['toiletCount'] / df['bathroomCount']
df.boxplot(column='price', by='toilet_efficiency', figsize=(20, 6))
plt.xticks(rotation=90)  # Vertical
plt.title('Price Distribution by Toilet/Bathroom Ratio')

In [ ]:
df['size_category'] = pd.cut(df['habitableSurface'], bins=[0, 50, 100, 150, 200, 500], labels=['Tiny', 'Small', 'Medium', 'Large', 'Mansion'])
df.groupby(['size_category', 'type_encoded'])['price'].mean().unstack().plot(kind='bar', figsize=(12, 6))
plt.title('Average Price by Size Category and Property Type')
plt.xticks(rotation=45)

In [ ]:
postal_prices = df.groupby('postCode')['price'].median().sort_values(ascending=False)
plt.figure(figsize=(15, 8))
postal_prices.head(20).plot(kind='bar')
plt.title('Top 20 Most Expensive Postal Codes (Median Price)')
plt.xticks(rotation=90)

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['price'] < Q1 - 1.5*IQR) | (df['price'] > Q3 + 1.5*IQR)]
print(f"Found {len(outliers)} outliers out of {len(df)} properties")
outliers[['habitableSurface', 'price', 'bedroomCount', 'postCode']].head(10)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import io

# Step 1: Download Belgian postal code coordinates
def download_belgian_postal_codes():
    try:
        # Direct download from GeoNames
        url = "https://download.geonames.org/export/zip/BE.zip"
        response = requests.get(url)
        
        if response.status_code == 200:
            # Read the zip file content
            import zipfile
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                with z.open('BE.txt') as f:
                    # GeoNames format: country code, postal code, place name, admin name1, admin code1, admin name2, admin code2, admin name3, admin code3, latitude, longitude, accuracy
                    postal_coords = pd.read_csv(f, sep='\t', header=None, 
                                              names=['country_code', 'postal_code', 'place_name', 
                                                   'admin_name1', 'admin_code1', 'admin_name2', 'admin_code2',
                                                   'admin_name3', 'admin_code3', 'latitude', 'longitude', 'accuracy'])
                    return postal_coords[['postal_code', 'place_name', 'latitude', 'longitude']]
        else:
            print("Could not download data automatically. Please download manually.")
            return None
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None



# Step 2: Create the heatmap
def create_belgian_price_heatmap(df):

        # Download postal code coordinates
    print("Downloading Belgian postal code coordinates...")
    postal_coords = download_belgian_postal_codes()
    
    if postal_coords is None:
        raise Exception("Could not download postal code data. Please check your internet connection or download manually from https://download.geonames.org/export/zip/BE.zip")
    
    # Calculate price statistics by postal code
    postal_prices = df.groupby('postCode').agg({
        'price': ['median', 'mean', 'count']
    }).round(0)
    
    # Flatten column names
    postal_prices.columns = ['median_price', 'mean_price', 'property_count']
    postal_prices = postal_prices.reset_index()
    
    # Merge with coordinates
    postal_coords['postCode'] = postal_coords['postal_code']  # Match column names
    map_data = postal_prices.merge(postal_coords, on='postCode', how='inner')
    
    # Remove outliers for better visualization
    Q1 = map_data['median_price'].quantile(0.25)
    Q3 = map_data['median_price'].quantile(0.75)
    IQR = Q3 - Q1
    map_data = map_data[
        (map_data['median_price'] >= Q1 - 1.5 * IQR) & 
        (map_data['median_price'] <= Q3 + 1.5 * IQR)
    ]
    
    # Create the interactive heatmap
    fig = px.scatter_mapbox(
        map_data,
        lat="latitude",
        lon="longitude",
        color="median_price",
        size="property_count",
        hover_name="place_name",
        hover_data={
            "postCode": True,
            "median_price": ":€,.0f",
            "mean_price": ":€,.0f", 
            "property_count": True,
            "latitude": False,
            "longitude": False
        },
        color_continuous_scale="inferno",  # Plasma for better visibility
        size_max=20,  
        zoom=7,
        center={"lat": 50.8503, "lon": 4.3517},  # Center on Brussels
        mapbox_style="open-street-map",  # Changed from open-street-map to light background
        title=" Belgian Real Estate Prices by Postal Code<br><sub>Size = Number of Properties, Color = Median Price</sub>",
        labels={"median_price": "Median Price (€)", "property_count": "Properties"}
    )
    
    # Customize the layout
    fig.update_layout(
        height=700,
        font=dict(size=12),
        title_font_size=16,
        coloraxis_colorbar=dict(
            title="Median Price (€)",
            title_font_size=14,
            tickformat="€,.0f"
        )
    )
    
    return fig

# Step 3: Usage example
if __name__ == "__main__":
   
    fig = create_belgian_price_heatmap(df)
    fig.show()
    
    fig.write_html("belgian_real_estate_heatmap.html")
    
   